In [1]:
import  numpy as np

# Часть 1

## 2.1 Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [2]:
n, k, d = 7, 4, 3
I_k = np.eye(k, dtype=int)
X = np.array([
        [1, 1, 0],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1]
    ])
G = np.hstack((I_k,X))
print(f'порождающая матрица линейного кода (7, 4, 3):\n{G}')

порождающая матрица линейного кода (7, 4, 3):
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 0 1 1]
 [0 0 0 1 1 1 1]]


## 2.2 Сформировать проверочную матрицу на основе порождающей

In [3]:
I_n_k = np.eye(n - k, dtype=int)

H = np.vstack((X, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## 2.3 Сформировать таблицу синдромов для всех однократных ошибок

In [4]:
def calculate_syndrome(E, H):
    """
    Вычисляет синдром для данного вектора ошибки и матрицы H.
    """
    return tuple((E @ H) % 2)

def build_syndrome_table(H):
    """
    Строит таблицу синдромов для матрицы H.
    """
    n = H.shape[0]
    syndromes = {}
    for i in range(n):
        E = np.zeros(n, dtype=int)
        E[i] = 1
        S = calculate_syndrome(E, H)
        syndromes[S] = tuple(E)
    return syndromes
s_t = build_syndrome_table(H)
print("Syndrome\tError vec")
for syndrome in s_t:
    print(f"{syndrome}\t{s_t[syndrome]}")

Syndrome	Error vec
(1, 1, 0)	(1, 0, 0, 0, 0, 0, 0)
(1, 0, 1)	(0, 1, 0, 0, 0, 0, 0)
(0, 1, 1)	(0, 0, 1, 0, 0, 0, 0)
(1, 1, 1)	(0, 0, 0, 1, 0, 0, 0)
(1, 0, 0)	(0, 0, 0, 0, 1, 0, 0)
(0, 1, 0)	(0, 0, 0, 0, 0, 1, 0)
(0, 0, 1)	(0, 0, 0, 0, 0, 0, 1)


## 2.4 Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова


In [5]:
def correct_errors(parity_check_matrix, received_word, syndromes):
    """Исправляет ошибки в принятом кодовом слове с помощью таблицы синдромов."""
    syndrome = tuple(np.dot(parity_check_matrix.T, received_word) % 2)
    if syndrome in syndromes:
        error_pattern = np.array(syndromes[syndrome], dtype=int)
        fixed_word = (received_word + error_pattern) % 2
        return fixed_word, error_pattern
    for i in range(len(received_word)):
        for j in range(i + 1, len(received_word)):  # Избегаем повторяющихся пар
            # Генерируем шаблон ошибки для текущей пары индексов
            error_pattern = np.zeros_like(received_word)
            error_pattern[i] = 1
            error_pattern[j] = 1

            # Вычисляем новый синдром
            new_received_word = (received_word + error_pattern) % 2
            new_syndrome = tuple(np.dot(parity_check_matrix.T, new_received_word) % 2)

            # Проверяем новый синдром
            if new_syndrome in syndromes:
                fixed_word = (received_word + error_pattern) % 2
                return fixed_word, error_pattern
    # Если ошибки не обнаружены, возвращаем исходное слово
    return received_word, None

In [6]:
original_msg = np.random.randint(0, 2, size=k)
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
# внесем однократную ошибку
pos = np.random.randint(len(encoded_word))
corrupted_word = np.array([bit ^ (i == pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print(f"Исправленное сообщение: {corrected_word}")
if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")
print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")


Оригинальное сообщение: [1 0 0 1]
Кодированное сообщение: [1 0 0 1 0 0 1]
Сообщение с ошибкой в позиции 5: [1 0 0 1 0 1 1]
Исправленное сообщение: [1 0 0 1 0 0 1]
Обнаруженная ошибка: [0 0 0 0 0 1 0]
Проверка на равенство отправленного слова и полученного: True


## 2.5. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [7]:

print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем двукратную ошибку
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print(f"Исправленное сообщение: {corrected_word}")
if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")
print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 0 1]
Кодированное сообщение: [1 0 0 1 0 0 1]
Сообщение с ошибкой в позиции [2 4]: [1 0 1 1 1 0 1]
Исправленное сообщение: [1 0 1 0 1 0 1]
Обнаруженная ошибка: [0 0 0 1 0 0 0]
Проверка на равенство отправленного слова и полученного: False


# Часть 2

## 2.6. Сформировать порождающую матрицу линейного кода (n, k, 5).

In [8]:
# Параметры кода
n = 9   # длина кодового слова
k = 4   # длина информационного слова
n_k = n - k

In [9]:
# Примерная матрица P
P = np.array([[1, 1, 1, 1, 0],
              [1, 1, 0, 0, 1],
              [1, 0, 1, 0, 1],
              [0, 1, 1, 0, 1]])

In [10]:
I_k = np.eye(k, dtype=int)
G = np.hstack((I_k, P))
print("Порождающая матрица G:\n", G)

Порождающая матрица G:
 [[1 0 0 0 1 1 1 1 0]
 [0 1 0 0 1 1 0 0 1]
 [0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 1]]


## 2.7 Сформировать проверочную матрицу на основе порождающей.

In [11]:
# Проверочная матрица H
I_n_k = np.eye(n_k, dtype=int)
H = np.vstack((P, I_n_k))
print("Проверочная матрица H:\n", H)

Проверочная матрица H:
 [[1 1 1 1 0]
 [1 1 0 0 1]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


## 2.8 Сформировать таблицу синдромов для всех однократных и двукратных ошибок.

In [12]:
def build_syndrome_table(H):
    n = H.shape[1]  # Количество столбцов (размер кодового слова)

    syndromes = {}

    # Однократные ошибки
    for i in range(n):
        E = np.zeros(n, dtype=int)
        E[i] = 1
        S = calculate_syndrome(E, H.T)
        syndromes[S] = tuple(E)

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            E = np.zeros(n, dtype=int)
            E[i] = 1
            E[j] = 1
            S = calculate_syndrome(E, H.T)
            syndromes[S] = tuple(E)

    return syndromes

# Пример проверочной матрицы H (замените H на вашу матрицу
s_t = build_syndrome_table(H.T)
print("Syndrome\t\tError vec")
for syndrome in s_t:
    print(f"{syndrome}\t\t{s_t[syndrome]}")


Syndrome		Error vec
(1, 1, 1, 1, 0)		(1, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 0, 0, 1)		(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 1, 0, 1)		(0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 1, 1, 0, 1)		(0, 0, 0, 1, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0)		(0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 1, 0, 0, 0)		(0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 1, 0, 0)		(0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 1, 0)		(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 1)		(0, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 1, 1, 1)		(1, 1, 0, 0, 0, 0, 0, 0, 0)
(0, 1, 0, 1, 1)		(1, 0, 1, 0, 0, 0, 0, 0, 0)
(1, 0, 0, 1, 1)		(1, 0, 0, 1, 0, 0, 0, 0, 0)
(0, 1, 1, 1, 0)		(1, 0, 0, 0, 1, 0, 0, 0, 0)
(1, 0, 1, 1, 0)		(1, 0, 0, 0, 0, 1, 0, 0, 0)
(1, 1, 0, 1, 0)		(1, 0, 0, 0, 0, 0, 1, 0, 0)
(1, 1, 1, 0, 0)		(1, 0, 0, 0, 0, 0, 0, 1, 0)
(1, 1, 1, 1, 1)		(1, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0)		(0, 0, 0, 0, 0, 1, 1, 0, 0)
(1, 0, 1, 0, 0)		(0, 0, 0, 0, 1, 0, 1, 0, 0)
(0, 1, 0, 0, 1)		(0, 0, 0, 0, 0, 1, 0, 0, 1)
(1, 0, 0, 0, 1)		(0, 0, 0, 0, 1, 0, 0, 0, 1)
(1, 1, 1, 0, 1)		(0, 0, 0, 1, 1, 0,

## 2.9. Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [13]:
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")

pos = np.random.choice(len(encoded_word), size=1, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")
corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print(f"Исправленное сообщение: {corrected_word}")
if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")
print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 0 1]
Кодированное сообщение: [1 0 0 1 1 0 0 1 1]
Сообщение с ошибкой в позиции [2]: [1 0 1 1 1 0 0 1 1]
Исправленное сообщение: [1 0 0 1 1 0 0 1 1]
Обнаруженная ошибка: [0 0 1 0 0 0 0 0 0]
Проверка на равенство отправленного слова и полученного: True


## 2.10. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [14]:
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")
corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print(f"Исправленное сообщение: {corrected_word}")
if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")
print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 0 1]
Кодированное сообщение: [1 0 0 1 1 0 0 1 1]
Сообщение с ошибкой в позиции [0 6]: [0 0 0 1 1 0 1 1 1]
Исправленное сообщение: [1 0 0 1 1 0 0 1 1]
Обнаруженная ошибка: [1 0 0 0 0 0 1 0 0]
Проверка на равенство отправленного слова и полученного: True


## 2.11. Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [15]:
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
pos = np.random.choice(len(encoded_word), size=3, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")
corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print(f"Исправленное сообщение: {corrected_word}")
if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")
print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [1 0 0 1]
Кодированное сообщение: [1 0 0 1 1 0 0 1 1]
Сообщение с ошибкой в позиции [2 1 8]: [1 1 1 1 1 0 0 1 0]
Исправленное сообщение: [1 1 1 0 1 0 0 1 0]
Обнаруженная ошибка: [0 0 0 1 0 0 0 0 0]
Проверка на равенство отправленного слова и полученного: False
